# Supervised Learning for Energy Efficiency
Author: Andrew Farell

In this project, I focus on predicting **heating and cooling loads** for residential buildings. My objective is to explore how different building features (like shape, surface area, and glazing) affect energy demands. **If I am successful, this model could help designers and architects optimize for efficiency.**

---

## Problem Definition

I’m working with the **ENB2012 dataset**, which includes 768 building configurations simulated in Ecotect. Each configuration has eight features (X1–X8) and two target variables:

- **X1**: Relative Compactness  
- **X2**: Surface Area  
- **X3**: Wall Area  
- **X4**: Roof Area  
- **X5**: Overall Height  
- **X6**: Orientation  
- **X7**: Glazing Area  
- **X8**: Glazing Area Distribution  

The two targets are:

1. **Y1** (Heating Load)  
2. **Y2** (Cooling Load)  

This is a supervised learnign task, i.e. I am trying to predict Y1 and Y2 from X1-X8.


## Exploratory Data Analysis

I'm going to first explore the data to see how the features and targets behave. The outputs of this analysis are in the `outputs/` folder, along with a file called `analysis.json` that has key statistical information.

In [3]:
import os
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

data = pd.read_excel("ENB2012_data.xlsx")
os.makedirs("outputs", exist_ok=True)

numeric_describe = data.describe().T

missing_summary = data.isna().sum().to_dict()

dtypes_summary = {col: str(dtype) for col, dtype in data.dtypes.items()}

corr = data.corr()
plt.figure(figsize=(8,6))
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Heatmap of All Features and Targets")
plt.tight_layout()
heatmap_path = os.path.join("outputs", "correlation_heatmap.png")
plt.savefig(heatmap_path, dpi=300)
plt.close()

for col in data.columns:
    plt.figure()
    sns.histplot(data[col], kde=True, color="blue", edgecolor="black")
    plt.title(f"Histogram of {col}")
    plt.xlabel(col)
    plt.ylabel("Frequency")
    plt.tight_layout()
    hist_path = os.path.join("outputs", f"{col}_hist.png")
    plt.savefig(hist_path, dpi=300)
    plt.close()

target_columns = ["Y1", "Y2"]
for target in target_columns:
    plt.figure()
    sns.boxplot(y=data[target], color="green")
    plt.title(f"Box Plot of {target}")
    plt.ylabel(target)
    plt.tight_layout()
    boxplot_path = os.path.join("outputs", f"{target}_boxplot.png")
    plt.savefig(boxplot_path, dpi=300)
    plt.close()

analysis_dict = {
    "numeric_describe": numeric_describe.to_dict(),
    "missing_values": missing_summary,
    "data_types": dtypes_summary,
    "correlation_matrix": corr.to_dict()
}

analysis_path = os.path.join("outputs", "analysis.json")
with open(analysis_path, "w", encoding="utf-8") as f:
    json.dump(analysis_dict, f, indent=4)

print("EDA completed successfully. Results saved to 'outputs/' directory.")

EDA completed successfully. Results saved to 'outputs/' directory.


### 1. Data Integrity and Basic Statistics

- **No Missing Values**: I confirmed that none of the 768 rows contain missing data.  
- **Data Types**: Most features are floats (e.g., X1, X2, etc.), while X6 and X8 are integers. This matches the dataset’s official description.  
- **Descriptive Stats**: Here are some highlights (more details can be found in `analysis.json`):  
  - **Relative Compactness (X1)** varies from 0.62 to 0.98 (mean ~0.76).  
  - **Surface Area (X2)** ranges from 514.5 to 808.5, averaging around 671.71.  
  - **Heating Load (Y1)** spans 6.01 to 43.1 (mean ~22.31).  
  - **Cooling Load (Y2)** goes from 10.9 to 48.03 (mean ~24.59).  

### 2. Univariate Distributions

I created histograms for each feature and target to visualize their distributions. For example:

![Histogram of X1](outputs/X1_hist.png)

Similar images for X2 through Y2 (e.g., `X2_hist.png`, `Y1_hist.png`, etc.) are located in the `outputs/` directory. These plots help me see how each variable is spread out (e.g., X1 peaks near 0.75, X2 clusters around ~670 m²).

### 3. Target Variable Box Plots

To spot any outliers, I also plotted box plots for **Heating Load (Y1)** and **Cooling Load (Y2)**:

![Y1 Box Plot](outputs/Y1_boxplot.png)
![Y2 Box Plot](outputs/Y2_boxplot.png)

The box plots indicate that a subset of buildings require substantially higher energy, which likely ties to differences in surface area, height, orientation, or other design aspects.

### 4. Correlation Analysis

Finally, I examined feature correlations and how they relate to the targets:

![Correlation Heatmap](outputs/correlation_heatmap.png)

Notable observations include:
- **X5 (Overall Height)** has a strong positive correlation with both Y1 (~0.89) and Y2 (~0.90). Taller buildings often need more energy for heating and cooling.  
- **X4 (Roof Area)** is strongly negatively correlated with both Y1 and Y2 (~-0.86 each). This likely reflects interactions between building height, roof surface, and thermal transfers.  
- **X1 (Relative Compactness)** also correlates well with Y1 and Y2, suggesting that more compact designs can reduce energy demands.


## Modeling and Results

### 1. Model Definitions

I tested two types of models:

1. **Ridge Regression**  
   This is a linear regression model with $\ell_2$ regularization. I used a pipeline to **scale the features** and fit a Ridge regressor, exploring multiple values of the regularization parameter $\alpha$.

2. **XGBoost**  
   XGBoost is a gradient boosting library that can handle complex, nonlinear relationships. I tuned:
   - **learning_rate**: A step size parameter that helps avoid overfitting,  
   - **max_depth**: The maximum depth of each decision tree,  
   - **n_estimators**: The total number of boosting rounds (trees).

### 2. Loss Function and Metrics

Both models aim to **minimize Mean Squared Error (MSE)**:

$$
\mathrm{MSE} \;=\; \frac{1}{n}\sum_{i=1}^n (\hat{y}_i - y_i)^2,
$$

where $y_i$ is the true heating or cooling load, and $\hat{y}_i$ is my model’s prediction. I used **5-fold cross-validation** to find the best hyperparameters and then evaluated final performance on a separate test set.

### 3. Training Procedure

1. **Data Splitting**: I reserved 20% of the data for final testing to ensure the models don’t overfit by seeing all samples during training.  
2. **Scaling and Fitting**: I standardized the features (subtracting mean, dividing by std) and then trained each model on the remaining 80% of data.  
3. **Hyperparameter Grid Search**: I systematically varied $\alpha$ (for Ridge) and learning_rate, max_depth, n_estimators (for XGBoost) across a range of values. The combination that achieved the lowest cross-validation MSE was then refit on the full training set.

In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.base import clone
from xgboost import XGBRegressor

os.makedirs("outputs", exist_ok=True)

df = pd.read_excel("ENB2012_data.xlsx")
X = df[["X1", "X2", "X3", "X4", "X5", "X6", "X7", "X8"]]
y1 = df["Y1"]
y2 = df["Y2"]

X_train, X_test, y1_train, y1_test = train_test_split(
    X, y1, test_size=0.2, random_state=42
)
_, _, y2_train, y2_test = train_test_split(
    X, y2, test_size=0.2, random_state=42
)

ridge_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("ridge", Ridge())
])
ridge_params = {
    "ridge__alpha": [0.01, 0.1, 1.0, 10.0, 100.0]
}

xgb_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("xgb", XGBRegressor(objective="reg:squarederror"))
])
xgb_params = {
    "xgb__learning_rate": [0.01, 0.1],
    "xgb__max_depth": [3, 5],
    "xgb__n_estimators": [100, 200]
}

ridge_results_df = pd.DataFrame()
xgb_results_df = pd.DataFrame()

def full_grid_search(pipeline, param_grid, X_tr, y_tr, X_te, y_te, model_name, target_name):
    grid = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        scoring="neg_mean_squared_error",
        cv=5,
        verbose=0,
        n_jobs=-1
    )
    grid.fit(X_tr, y_tr)

    cv_res = grid.cv_results_

    rows = []
    for i, param_combo in enumerate(cv_res["params"]):
        row_dict = {}
        row_dict["model"] = model_name
        row_dict["target"] = target_name

        for k, v in param_combo.items():
            row_dict[k] = v

        mean_cv_neg_mse = cv_res["mean_test_score"][i]
        std_cv_neg_mse = cv_res["std_test_score"][i]
        rank = cv_res["rank_test_score"][i]

        row_dict["mean_cv_mse"] = -mean_cv_neg_mse
        row_dict["std_cv_mse"] = std_cv_neg_mse
        row_dict["rank_test_score"] = rank

        pipe_copy = clone(pipeline)
        pipe_copy.set_params(**param_combo)
        pipe_copy.fit(X_tr, y_tr)
        preds = pipe_copy.predict(X_te)
        test_mse = np.mean((preds - y_te) ** 2)
        row_dict["test_mse"] = test_mse

        rows.append(row_dict)

    return pd.DataFrame(rows)

ridge_y1_df = full_grid_search(
    ridge_pipeline, ridge_params,
    X_train, y1_train, X_test, y1_test,
    "Ridge", "Y1"
)
ridge_results_df = pd.concat([ridge_results_df, ridge_y1_df], ignore_index=True)

ridge_y2_df = full_grid_search(
    ridge_pipeline, ridge_params,
    X_train, y2_train, X_test, y2_test,
    "Ridge", "Y2"
)
ridge_results_df = pd.concat([ridge_results_df, ridge_y2_df], ignore_index=True)

xgb_y1_df = full_grid_search(
    xgb_pipeline, xgb_params,
    X_train, y1_train, X_test, y1_test,
    "XGBoost", "Y1"
)
xgb_results_df = pd.concat([xgb_results_df, xgb_y1_df], ignore_index=True)

xgb_y2_df = full_grid_search(
    xgb_pipeline, xgb_params,
    X_train, y2_train, X_test, y2_test,
    "XGBoost", "Y2"
)
xgb_results_df = pd.concat([xgb_results_df, xgb_y2_df], ignore_index=True)

ridge_results_path = os.path.join("outputs", "ridge_results.csv")
xgb_results_path = os.path.join("outputs", "xgb_results.csv")

ridge_results_df.to_csv(ridge_results_path, index=False)
xgb_results_df.to_csv(xgb_results_path, index=False)

from sklearn.model_selection import learning_curve

def get_best_params(df_results):
    best_idx = df_results["mean_cv_mse"].idxmin()
    best_row = df_results.loc[best_idx].to_dict()
    param_dict = {}
    for k, v in best_row.items():
        if k.startswith("ridge__") or k.startswith("xgb__"):
            param_dict[k] = v
    return param_dict

def plot_learning_curves(estimator, X, y, title):
    train_sizes, train_scores, val_scores = learning_curve(
        estimator, X, y,
        cv=5, scoring="neg_mean_squared_error",
        train_sizes=np.linspace(0.1, 1.0, 5),
        shuffle=True, random_state=42,
        n_jobs=-1
    )

    train_mse = -train_scores
    val_mse = -val_scores

    train_mse_mean = np.mean(train_mse, axis=1)
    train_mse_std = np.std(train_mse, axis=1)
    val_mse_mean = np.mean(val_mse, axis=1)
    val_mse_std = np.std(val_mse, axis=1)

    plt.figure()
    plt.plot(train_sizes, train_mse_mean, marker='o', label='Training MSE')
    plt.fill_between(train_sizes,
                        train_mse_mean - train_mse_std,
                        train_mse_mean + train_mse_std,
                        alpha=0.2)
    plt.plot(train_sizes, val_mse_mean, marker='s', label='Validation MSE')
    plt.fill_between(train_sizes,
                        val_mse_mean - val_mse_std,
                        val_mse_mean + val_mse_std,
                        alpha=0.2)

    plt.title(f"Learning Curves: {title}")
    plt.xlabel("Training Set Size")
    plt.ylabel("MSE")
    plt.legend()
    plt.tight_layout()

best_ridge_y1 = get_best_params(ridge_results_df[ridge_results_df["target"]=="Y1"])
ridge_pipeline_best_y1 = Pipeline([
    ("scaler", StandardScaler()),
    ("ridge", Ridge())
])
ridge_pipeline_best_y1.set_params(**best_ridge_y1)

plot_learning_curves(ridge_pipeline_best_y1, X_train, y1_train, "Ridge (Best) - Y1")
plt.savefig(os.path.join("outputs", "learning_curve_ridge_y1.png"), dpi=300)
plt.close()

best_ridge_y2 = get_best_params(ridge_results_df[ridge_results_df["target"]=="Y2"])
ridge_pipeline_best_y2 = Pipeline([
    ("scaler", StandardScaler()),
    ("ridge", Ridge())
])
ridge_pipeline_best_y2.set_params(**best_ridge_y2)

plot_learning_curves(ridge_pipeline_best_y2, X_train, y2_train, "Ridge (Best) - Y2")
plt.savefig(os.path.join("outputs", "learning_curve_ridge_y2.png"), dpi=300)
plt.close()

best_xgb_y1 = get_best_params(xgb_results_df[xgb_results_df["target"]=="Y1"])
xgb_pipeline_best_y1 = Pipeline([
    ("scaler", StandardScaler()),
    ("xgb", XGBRegressor(objective="reg:squarederror"))
])
xgb_pipeline_best_y1.set_params(**best_xgb_y1)

plot_learning_curves(xgb_pipeline_best_y1, X_train, y1_train, "XGBoost (Best) - Y1")
plt.savefig(os.path.join("outputs", "learning_curve_xgb_y1.png"), dpi=300)
plt.close()

best_xgb_y2 = get_best_params(xgb_results_df[xgb_results_df["target"]=="Y2"])
xgb_pipeline_best_y2 = Pipeline([
    ("scaler", StandardScaler()),
    ("xgb", XGBRegressor(objective="reg:squarederror"))
])
xgb_pipeline_best_y2.set_params(**best_xgb_y2)

plot_learning_curves(xgb_pipeline_best_y2, X_train, y2_train, "XGBoost (Best) - Y2")
plt.savefig(os.path.join("outputs", "learning_curve_xgb_y2.png"), dpi=300)
plt.close()

print("All model results saved to 'ridge_results.csv' and 'xgb_results.csv'.")
print("Learning curves saved to 'outputs/'.")

All model results saved to 'ridge_results.csv' and 'xgb_results.csv'.
Learning curves saved to 'outputs/'.


### 4. Results Overview

I recorded **every** parameter combination’s average cross-validation MSE and test-set MSE in two CSV files:
- [`ridge_results.csv`](outputs/ridge_results.csv)  
- [`xgb_results.csv`](outputs/xgb_results.csv)  

These files list each tried hyperparameter setting, along with the corresponding errors. Checking them out lets you see which values of $\alpha$ or tree depth worked best.

### 5. Learning Curves

I also plotted learning curves for the best-performing parameter settings on each model and target:

![Ridge (Best) - Y1](outputs/learning_curve_ridge_y1.png)  
![Ridge (Best) - Y2](outputs/learning_curve_ridge_y2.png)  
![XGBoost (Best) - Y1](outputs/learning_curve_xgb_y1.png)  
![XGBoost (Best) - Y2](outputs/learning_curve_xgb_y2.png)

These graphs display **Training MSE** and **Validation MSE** as the training set size grows. A narrow gap between these lines usually indicates that the model generalizes well and isn’t overly complex.

### 6. Interpretation and Next Steps

- **Ridge Regression**: Offers a more straightforward, linear way to capture relationships. Its MSE is moderate is not good though. This shows that while some linear patterns exist, the problem likely has nonlinear relationships. I don't recommend using this model. 
- **XGBoost**: Tends to produce substantially lower errors for both heating (Y1) and cooling (Y2) tasks. This is the model I would recommend. It can handle nonlinear interactions between building parameters.

If you want a model that’s easier to interpret, a linear approach like Ridge may be appropriate. However, XGBoost is a better choice. 
